In [2]:
from __future__ import print_function
from collections import OrderedDict

import matplotlib.pyplot as plt
import numpy as np

from nn_transfer import transfer, util, p2f_trans

%matplotlib inline

Using TensorFlow backend.


ImportError: cannot import name 'p2f_trans' from 'nn_transfer' (/opt/anaconda3/lib/python3.7/site-packages/nn_transfer-0.1.0-py3.7.egg/nn_transfer/__init__.py)

## Step 1
Simply define your PyTorch model like usual, and create an instance of it.

In [34]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out
    
pytorch_network = LeNet()

## Step 2
Determine the names of the layers.

For the above model example it is very straightforward, but if you use param groups it may be a little more involved. To determine the names of the layers the next commands are useful:

In [35]:
# The most useful, just print the network
print(pytorch_network)

# Also useful: will only print those layers with params
state_dict = pytorch_network.state_dict()
print(util.state_dict_layer_names(state_dict))
for k,v in state_dict.items():
    print(k)
print(state_dict['conv1.weight'])
print(state_dict['conv1.weight'].shape)


LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
['conv1', 'conv2', 'fc1', 'fc2', 'fc3']
conv1.weight
conv1.bias
conv2.weight
conv2.bias
fc1.weight
fc1.bias
fc2.weight
fc2.bias
fc3.weight
fc3.bias
tensor([[[[ 0.1858,  0.0280,  0.1866,  0.0164, -0.0935],
          [-0.1696,  0.0197, -0.1052, -0.1162,  0.0364],
          [ 0.0444,  0.0057, -0.1944,  0.1153,  0.0736],
          [ 0.0470,  0.1928,  0.1636,  0.1727, -0.1349],
          [ 0.1477, -0.0953,  0.1393,  0.1589, -0.0610]]],


        [[[ 0.0333, -0.0613, -0.1085,  0.0499, -0.0678],
          [-0.1099, -0.1834, -0.1671,  0.0997,  0.1249],
          [-0.0817,  0.0724, -0.1199, -0.1299,  0.0485],
          [-0.0304,  0.0862,  0.1196,  0.1391,  0.0029],
          [-0.1875,  0.1

## Step 3
Define an equivalent Keras network. Use the built-in `name` keyword argument for each layer with params.

In [1]:
import paddle
import paddle.fluid as fluid
import numpy as np
from paddle.fluid.dygraph.nn import Conv2D, Pool2D, Linear, Conv2DTranspose
from paddle.fluid.dygraph.base import to_variable
# K.set_image_data_format('channels_first')
# 定义 LeNet 网络结构
class LeNet(fluid.dygraph.Layer):
    def __init__(self, num_classes=1):
        super(LeNet, self).__init__()

        # 创建卷积和池化层块，每个卷积层使用Sigmoid激活函数，后面跟着一个2x2的池化
        self.conv1 = Conv2D(num_channels=1, num_filters=6, filter_size=5, act='relu')
        self.pool1 = Pool2D(pool_size=2, pool_stride=2, pool_type='max')
        self.conv2 = Conv2D(num_channels=6, num_filters=16, filter_size=5, act='relu')
        self.pool2 = Pool2D(pool_size=2, pool_stride=2, pool_type='max')
        # 创建第3个卷积层
        self.conv3 = Conv2D(num_channels=16, num_filters=120, filter_size=4, act='relu')
        # 创建全连接层，第一个全连接层的输出神经元个数为64， 第二个全连接层输出神经元个数为分裂标签的类别数
        self.fc1 = Linear(input_dim=120, output_dim=64, act='relu')
        self.fc2 = Linear(input_dim=64, output_dim=num_classes)
    # 网络的前向计算过程
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = fluid.layers.reshape(x, [x.shape[0], -1])
        x = self.fc1(x)
        x = self.fc2(x)
        return x

with fluid.dygraph.guard():
    paddle_network = LeNet()
    print(paddle_network)
    state_dict = paddle_network.state_dict()
    # print(util.state_dict_layer_names(state_dict))
    for k, v in state_dict.items():
        print(k)
    print(state_dict['conv1.weight'])
    state_dict.numpy

conv1.weight
conv1.bias
conv2.weight
conv2.bias
conv3.weight
conv3.bias
fc1.weight
fc1.bias
fc2.weight
fc2.bias
name conv2d_0.w_0, dtype: VarType.FP32 shape: [6, 1, 5, 5] 	lod: {}
	dim: 6, 1, 5, 5
	layout: NCHW
	dtype: float
	data: [0.363508 0.338076 0.130551 -0.147174 -0.440823 0.149725 -0.198654 -0.582418 0.0707067 -0.10575 0.229078 -0.0669537 0.103903 0.300438 -0.86238 -0.435697 0.413513 0.0775742 -0.111762 0.153061 -0.174628 -0.178401 0.21513 0.0247442 -0.0769519 -0.410499 0.319113 -0.359963 -0.33709 0.105977 -0.00229228 0.149699 0.149706 -0.412457 -0.211419 0.119478 0.288736 0.644174 -0.43126 -0.0462208 0.128646 0.195964 -0.397832 -0.198501 0.701616 -0.149692 -0.215233 0.179813 -0.0870564 -0.420398 0.0267434 0.601046 -0.326398 0.345797 -0.196241 -0.0230152 0.375119 0.487588 -0.415174 0.0502467 -0.0335857 0.115494 0.0567903 0.328555 -0.0558925 -0.141201 0.239747 -0.532105 -0.357474 0.376572 0.134131 0.30405 0.146998 -0.198026 -0.0572155 -0.618962 0.337098 0.308037 -0.189114 -0.1153

AttributeError: 'collections.OrderedDict' object has no attribute 'numpy'

In [37]:
# def test_qat():
#     with fluid.dygraph.guard():
#         mnist = LeNet()
        # transform_qat(mnist)
        # adam = AdamOptimizer(
            # learning_rate=0.001, parameter_list=mnist.parameters())
        # train_reader = paddle.batch(
            # paddle.dataset.mnist.train(), batch_size=32, drop_last=True)

        # epoch_num = 0
        # for epoch in range(epoch_num):
        #     for batch_id, data in enumerate(train_reader()):
        #         dy_x_data = np.array(
        #             [x[0].reshape(1, 28, 28)
        #                 for x in data]).astype('float32')
        #         y_data = np.array(
        #             [x[1] for x in data]).astype('int64').reshape(-1, 1)

        #         img = fluid.dygraph.to_variable(dy_x_data)
        #         label = fluid.dygraph.to_variable(y_data)

        #         cost, acc = mnist(img, label)
        #         loss = fluid.layers.cross_entropy(cost, label)
        #         avg_loss = fluid.layers.mean(loss)
        #         avg_loss.backward()
        #         adam.minimize(avg_loss)
        #         mnist.clear_gradients()
        #         if batch_id % 100 == 0:
        # #             print("Loss at epoch {} step {}: {:}".format(
        # #                 epoch, batch_id, avg_loss.numpy()))

        # model_dict = mnist.state_dict()
        # print(model_dict['conv1.weight'])
        # for k, v in model_dict.items():
        #     # if isinstance(v, (Variable, core.VarBase)):
        #     #     model_dict[k] = v.numpy()
        #     # else:
        #     #     model_dict[k] = v
        #     print("", k)

        # fluid.save_dygraph(model_dict, "save_temp")

# test_qat()

## Step 4
Now simply convert!

In [38]:
# transfer.pytorch_to_paddle(keras_network, pytorch_network)
p2f_trans.pytorch_to_paddle(pytorch_network, paddle_network)


AttributeError: 'LeNet' object has no attribute 'save'

## Done!

Now let's check whether it was succesful. If it was, both networks should have the same output.

In [7]:
# Create dummy data
# data = torch.rand(6,1,32,32)
# data_keras = data.numpy()
# data_pytorch = Variable(data, requires_grad=False)

# # Do a forward pass in both frameworks
# keras_pred = keras_network.predict(data_keras)
# pytorch_pred = pytorch_network(data_pytorch).data.numpy()

# Create dummy data
data = torch.rand(6,1,32,32)
data_paddle = data.numpy()
data_pytorch = Variable(data, requires_grad=False)

# Do a forward pass in both frameworks
paddle_pred = paddle_network.predict(data_paddle)
pytorch_pred = pytorch_network(data_pytorch).data.numpy()

In [8]:
# assert keras_pred.shape == pytorch_pred.shape

# plt.axis('Off')
# plt.imshow(keras_pred)
# plt.show()
# plt.axis('Off')
# plt.imshow(pytorch_pred)
# plt.show()

They are the same, it works :)